In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [25]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
try:
    train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')
    test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')
    sample_submission_df = pd.read_csv('/kaggle/input/playground-series-s5e4/sample_submission.csv') # Also load sample submission for context

    print("--- Datasets loaded successfully ---")
    print("\n")

    # --- Display basic information about the training data ---
    print("--- Training Data ---")
    print("Shape (rows, columns):", train_df.shape)
    print("\nFirst 5 rows:")
    print(train_df.head())
    print("\nData types and non-null counts:")
    train_df.info()
    print("-" * 30)
    print("\n")

    # --- Display basic information about the test data ---
    print("--- Test Data ---")
    print("Shape (rows, columns):", test_df.shape)
    print("\nFirst 5 rows:")
    print(test_df.head())
    print("\nData types and non-null counts:")
    test_df.info()
    print("-" * 30)
    print("\n")

    # --- Display basic information about the sample submission ---
    print("--- Sample Submission ---")
    print("Shape (rows, columns):", sample_submission_df.shape)
    print("\nFirst 5 rows:")
    print(sample_submission_df.head())
    print("\nData types and non-null counts:")
    sample_submission_df.info()
    print("-" * 30)

except FileNotFoundError as e:
    print(f"Error loading files: {e}")
    print("Please ensure 'train.csv', 'test.csv', and 'sample_submission.csv' are in the same directory as the script.")

--- Datasets loaded successfully ---


--- Training Data ---
Shape (rows, columns): (750000, 12)

First 5 rows:
   id     Podcast_Name Episode_Title  Episode_Length_minutes       Genre  \
0   0  Mystery Matters    Episode 98                     NaN  True Crime   
1   1    Joke Junction    Episode 26                  119.80      Comedy   
2   2   Study Sessions    Episode 16                   73.90   Education   
3   3   Digital Digest    Episode 45                   67.17  Technology   
4   4      Mind & Body    Episode 86                  110.51      Health   

   Host_Popularity_percentage Publication_Day Publication_Time  \
0                       74.81        Thursday            Night   
1                       66.95        Saturday        Afternoon   
2                       69.97         Tuesday          Evening   
3                       57.22          Monday          Morning   
4                       80.07          Monday        Afternoon   

   Guest_Popularity_percentage  Nu

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           750000 non-null  int64  
 1   Podcast_Name                 750000 non-null  object 
 2   Episode_Title                750000 non-null  object 
 3   Episode_Length_minutes       662907 non-null  float64
 4   Genre                        750000 non-null  object 
 5   Host_Popularity_percentage   750000 non-null  float64
 6   Publication_Day              750000 non-null  object 
 7   Publication_Time             750000 non-null  object 
 8   Guest_Popularity_percentage  603970 non-null  float64
 9   Number_of_Ads                749999 non-null  float64
 10  Episode_Sentiment            750000 non-null  object 
 11  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 68.7+ MB


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [6]:
num_cols = [
    'Episode_Length_minutes',
    'Host_Popularity_percentage',
    'Guest_Popularity_percentage',
    'Number_of_Ads'
]

# 1. Correlations with the target
print("=== Numeric Feature Correlations with Listening_Time_minutes ===")
print(train[num_cols + ['Listening_Time_minutes']].corr()['Listening_Time_minutes'].sort_values(ascending=False))

# 2. Mean Listening Time by categorical feature
cat_cols = ['Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
for col in cat_cols:
    print(f"\n=== Top 5 {col} by Average Listening_Time ===")
    means = train.groupby(col)['Listening_Time_minutes'].mean().sort_values(ascending=False)
    print(means.head(5))


=== Numeric Feature Correlations with Listening_Time_minutes ===
Listening_Time_minutes         1.000000
Episode_Length_minutes         0.916749
Host_Popularity_percentage     0.050870
Guest_Popularity_percentage   -0.016014
Number_of_Ads                 -0.118337
Name: Listening_Time_minutes, dtype: float64

=== Top 5 Genre by Average Listening_Time ===
Genre
Music         46.578394
True Crime    46.042507
Health        45.741413
Education     45.736640
Technology    45.634749
Name: Listening_Time_minutes, dtype: float64

=== Top 5 Publication_Day by Average Listening_Time ===
Publication_Day
Tuesday      46.131411
Monday       45.969630
Wednesday    45.807177
Saturday     45.326775
Friday       45.206591
Name: Listening_Time_minutes, dtype: float64

=== Top 5 Publication_Time by Average Listening_Time ===
Publication_Time
Night        46.456655
Afternoon    45.525603
Morning      44.964415
Evening      44.761567
Name: Listening_Time_minutes, dtype: float64

=== Top 5 Episode_Sentimen

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 1. Prepare X and y
X = train[['Episode_Length_minutes']].copy()
y = train['Listening_Time_minutes']

# 2. Impute any missing episode lengths with the median
X['Episode_Length_minutes'].fillna(X['Episode_Length_minutes'].median(), inplace=True)

# 3. Train/validation split (80/20)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. Fit a simple Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

# 5. Predict & evaluate
preds = lr.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE (Episode_Length only):", rmse)


Validation RMSE (Episode_Length only): 13.504102080390092


/tmp/ipykernel_31/3722330492.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['Episode_Length_minutes'].fillna(X['Episode_Length_minutes'].median(), inplace=True)


In [9]:
X = train[num_cols].copy()
y = train['Listening_Time_minutes']

# 2. Impute missing values with each column’s median
for col in num_cols:
    X[col].fillna(X[col].median(), inplace=True)

# 3. Train/validation split (80/20)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. Fit Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

# 5. Predict & evaluate
preds = lr.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE (All numeric features):", rmse)

/tmp/ipykernel_31/1316226645.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].median(), inplace=True)


Validation RMSE (All numeric features): 13.360688984602454


In [10]:
X_num = train[num_cols].copy()
X_cat = train[cat_cols].copy()
y = train['Listening_Time_minutes']

# 2. Impute numeric missing values with median
for col in num_cols:
    X_num[col].fillna(X_num[col].median(), inplace=True)

# 3. One-hot encode categoricals (drop first to avoid collinearity)
X_cat = pd.get_dummies(X_cat, drop_first=True)

# 4. Combine
X = pd.concat([X_num, X_cat], axis=1)

# 5. Train/validation split (80/20)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 6. Fit Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

# 7. Predict & evaluate
preds = lr.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE (Numeric + One-Hot Cats):", rmse)

/tmp/ipykernel_31/3436928066.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_num[col].fillna(X_num[col].median(), inplace=True)


Validation RMSE (Numeric + One-Hot Cats): 13.34639777602109


In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define a grid of alphas
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

# Set up RidgeCV (5-fold internally, using negative MSE)
ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train, y_train)

print("Best alpha found:", ridge_cv.alpha_)

# Evaluate on validation
preds = ridge_cv.predict(X_valid)
rmse_ridge = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE (Ridge):", rmse_ridge)

Best alpha found: 100.0
Validation RMSE (Ridge): 13.346385279648436


In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Initialize and fit HGB
hgb = HistGradientBoostingRegressor(
    max_iter=200,
    learning_rate=0.1,
    max_leaf_nodes=31,
    random_state=42
)
hgb.fit(X_train, y_train)

# 3. Predict & evaluate
preds = hgb.predict(X_valid)
rmse_hgb = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE (HistGradientBoosting):", rmse_hgb)

Validation RMSE (HistGradientBoosting): 13.051563326842889


In [17]:
df = train.copy()

# 2. Feature engineering
#   a) Episode number from title (e.g. "Episode 98" → 98)
df['Episode_Number'] = df['Episode_Title'].str.extract(r'(\d+)').astype(float)

#   b) Frequency encoding of Podcast_Name
freq = df['Podcast_Name'].value_counts(normalize=True)
df['Podcast_Name_freq'] = df['Podcast_Name'].map(freq)

# 3. Prepare feature matrix
num_cols = [
    'Episode_Length_minutes',
    'Host_Popularity_percentage',
    'Guest_Popularity_percentage',
    'Number_of_Ads',
    'Episode_Number',
    'Podcast_Name_freq'
]

cat_cols = ['Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

# 4. Impute numeric
X_num = df[num_cols].copy()
for c in num_cols:
    X_num[c].fillna(X_num[c].median(), inplace=True)

# 5. One-hot encode categoricals
X_cat = pd.get_dummies(df[cat_cols], drop_first=True)

# 6. Combine X, and target y
X = pd.concat([X_num, X_cat], axis=1)
y = df['Listening_Time_minutes']

# 7. Train/validation split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 8. Re-train HGB
hgb = HistGradientBoostingRegressor(
    max_iter=200,
    learning_rate=0.1,
    max_leaf_nodes=31,
    random_state=42
)
hgb.fit(X_train, y_train)

# 9. Evaluate
preds = hgb.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE (with Episode_Number & Podcast_Name_freq):", rmse)


/tmp/ipykernel_31/2606388716.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_num[c].fillna(X_num[c].median(), inplace=True)


Validation RMSE (with Episode_Number & Podcast_Name_freq): 13.03726407248093


In [21]:
df = train.copy()
df['Episode_Number'] = df['Episode_Title'].str.extract(r'(\d+)').astype(float)
freq = df['Podcast_Name'].value_counts(normalize=True)
df['Podcast_Name_freq'] = df['Podcast_Name'].map(freq)

num_cols = [
    'Episode_Length_minutes',
    'Host_Popularity_percentage',
    'Guest_Popularity_percentage',
    'Number_of_Ads',
    'Episode_Number',
    'Podcast_Name_freq'
]
cat_cols = ['Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

# Impute numerics
X_num = df[num_cols].copy()
for c in num_cols:
    X_num[c].fillna(X_num[c].median(), inplace=True)

# One-hot encode cats
X_cat = pd.get_dummies(df[cat_cols], drop_first=True)

# 2. TF-IDF on Episode_Title
tfidf = TfidfVectorizer(max_features=200, ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(df['Episode_Title'])

# 3. Combine all features into a sparse matrix
X_sparse = hstack([
    X_tfidf,
    np.array(X_num),       # dense to sparse automatically
    np.array(X_cat)
])

y = df['Listening_Time_minutes']

# 4. Train/validation split
X_train, X_valid, y_train, y_valid = train_test_split(
    X_sparse, y, test_size=0.2, random_state=42
)

# WARNING: this can blow up your memory if X is large!
X_train_d = X_train.toarray()
X_valid_d = X_valid.toarray()

ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train_d, y_train)
print("Best alpha:", ridge_cv.alpha_)
preds = ridge_cv.predict(X_valid_d)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print("RMSE (dense RidgeCV):", rmse)

# 6. Evaluate
preds = ridge_cv.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE (with TF-IDF on Title):", rmse)

/tmp/ipykernel_31/3076631079.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_num[c].fillna(X_num[c].median(), inplace=True)


Best alpha: 1.0
RMSE (dense RidgeCV): 13.338399524682325
Validation RMSE (with TF-IDF on Title): 13.338399524682325


In [28]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# ─── 1) Rebuild engineered feature matrix (no TF-IDF here) ────────────────
df = train.copy()
df['Episode_Number'] = df['Episode_Title'].str.extract(r'(\d+)').astype(float)
freq = df['Podcast_Name'].value_counts(normalize=True)
df['Podcast_Name_freq'] = df['Podcast_Name'].map(freq)

num_cols = [
    'Episode_Length_minutes',
    'Host_Popularity_percentage',
    'Guest_Popularity_percentage',
    'Number_of_Ads',
    'Episode_Number',
    'Podcast_Name_freq'
]
cat_cols = ['Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

# Impute numerics
X_num = df[num_cols].fillna(df[num_cols].median())

# One-hot encode cats
X_cat = pd.get_dummies(df[cat_cols], drop_first=True)

# Final feature matrix and target
X = pd.concat([X_num, X_cat], axis=1)
y = df['Listening_Time_minutes']

# Train/val split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# ─── 2) RandomizedSearch over HGB params ────────────────────────────────────

hgb = HistGradientBoostingRegressor(random_state=42)

param_dist = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_iter': [100, 200, 300],
    'max_leaf_nodes': [15, 31, 63],
    'min_samples_leaf': [20, 50, 100],
    'l2_regularization': [0.0, 0.1, 1.0]
}

search = RandomizedSearchCV(
    hgb,
    param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',
    cv=3,
    random_state=42,
    n_jobs=-1
)
search.fit(X_train, y_train)

print("Best HGB params:", search.best_params_)

# Evaluate on held-out validation
best_hgb = search.best_estimator_
preds = best_hgb.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE (tuned HGB):", rmse)

Best HGB params: {'min_samples_leaf': 100, 'max_leaf_nodes': 63, 'max_iter': 300, 'learning_rate': 0.1, 'l2_regularization': 1.0}
Validation RMSE (tuned HGB): 12.972773875532505


In [31]:
from scipy.sparse import csr_matrix

df = train.copy()

# a) Engineered tabular features (for HGB)
df['Episode_Number'] = df['Episode_Title'].str.extract(r'(\d+)').astype(float)
freq = df['Podcast_Name'].value_counts(normalize=True)
df['Podcast_Name_freq'] = df['Podcast_Name'].map(freq)

num_cols = [
    'Episode_Length_minutes',
    'Host_Popularity_percentage',
    'Guest_Popularity_percentage',
    'Number_of_Ads',
    'Episode_Number',
    'Podcast_Name_freq'
]
cat_cols = ['Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

X_tab = pd.concat([
    df[num_cols].fillna(df[num_cols].median()),
    pd.get_dummies(df[cat_cols], drop_first=True)
], axis=1)

# b) Sparse text features (for SGD)
tfidf = TfidfVectorizer(max_features=200, ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(df['Episode_Title'])

# c) Stack into one sparse matrix—cast tabular to float64 so scipy accepts it
X_sparse = hstack([
    X_tfidf,
    csr_matrix(X_tab.values.astype(np.float64))
])

y = df['Listening_Time_minutes'].values

# ─── 2) Train / validation split ────────────────────────────────────────────
X_tab_train, X_tab_valid, X_sparse_train, X_sparse_valid, y_train, y_valid = train_test_split(
    X_tab, X_sparse, y, test_size=0.2, random_state=42
)

# ─── 3) Level-1 models ────────────────────────────────────────────────────────

# (a) Tuned HGB on tabular data
hgb = HistGradientBoostingRegressor(
    max_iter=300,
    learning_rate=0.1,
    max_leaf_nodes=63,
    min_samples_leaf=100,
    l2_regularization=1.0,
    random_state=42
)
hgb.fit(X_tab_train, y_train)
pred_tab_train = hgb.predict(X_tab_train).reshape(-1, 1)
pred_tab_valid = hgb.predict(X_tab_valid).reshape(-1, 1)

# (b) SGDRegressor on sparse data
sgd = SGDRegressor(penalty='l2', alpha=0.01, random_state=42, max_iter=1000, tol=1e-3)
sgd.fit(X_sparse_train, y_train)
pred_sp_train = sgd.predict(X_sparse_train).reshape(-1, 1)
pred_sp_valid = sgd.predict(X_sparse_valid).reshape(-1, 1)

# ─── 4) Meta-model training ───────────────────────────────────────────────────
X_meta_train = np.hstack([pred_tab_train, pred_sp_train])
X_meta_valid = np.hstack([pred_tab_valid, pred_sp_valid])

meta = LinearRegression()
meta.fit(X_meta_train, y_train)

# Final ensemble evaluation
pred_ens = meta.predict(X_meta_valid)
rmse_ens = np.sqrt(mean_squared_error(y_valid, pred_ens))
print("Validation RMSE (Stacked Ensemble):", rmse_ens)

Validation RMSE (Stacked Ensemble): 12.97380209910441


In [32]:
train['Episode_Number'] = train['Episode_Title'].str.extract(r'(\d+)').astype(float)
test ['Episode_Number'] = test ['Episode_Title'].str.extract(r'(\d+)').astype(float)

# b) Podcast_Name frequency (using train distribution; unknowns → 0)
freq = train['Podcast_Name'].value_counts(normalize=True)
train['Podcast_Name_freq'] = train['Podcast_Name'].map(freq)
test ['Podcast_Name_freq'] = test ['Podcast_Name'].map(freq).fillna(0)

# c) Define columns
num_cols = [
    'Episode_Length_minutes',
    'Host_Popularity_percentage',
    'Guest_Popularity_percentage',
    'Number_of_Ads',
    'Episode_Number',
    'Podcast_Name_freq'
]
cat_cols = ['Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

# d) Impute numerics on train (medians) and apply same to test
medians = train[num_cols].median()
X_train_num = train[num_cols].fillna(medians)
X_test_num  = test [num_cols].fillna(medians)

# e) One-hot encode categoricals (drop_first for consistency)
X_train_cat = pd.get_dummies(train[cat_cols], drop_first=True)
# Remember the dummy-column names
cat_dummy_cols = X_train_cat.columns

# For test, align to the same set of dummy columns
X_test_cat = pd.get_dummies(test[cat_cols], drop_first=True)
X_test_cat = X_test_cat.reindex(columns=cat_dummy_cols, fill_value=0)

# f) Final feature matrices
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test  = pd.concat([X_test_num,  X_test_cat ], axis=1)
y_train = train['Listening_Time_minutes']

# ─── 3) Train Tuned HGB on Full Data ────────────────────────────────────────
model = HistGradientBoostingRegressor(
    max_iter=300,
    learning_rate=0.1,
    max_leaf_nodes=63,
    min_samples_leaf=100,
    l2_regularization=1.0,
    random_state=42
)
model.fit(X_train, y_train)

# ─── 4) Predict on Test and Save Submission ─────────────────────────────────
preds = model.predict(X_test)
submission = pd.DataFrame({
    'id': test['id'],
    'Listening_Time_minutes': preds
})
submission.to_csv('submission.csv', index=False)
print("➡️  submission.csv created with", len(submission), "rows")

➡️  submission.csv created with 250000 rows
